<a href="https://colab.research.google.com/github/Sripriya07/Q-Learning-Reinforcement-Learning-/blob/master/22_Reinforcement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from IPython.display import clear_output

In [0]:
import gym
import random
import numpy as np
import time

In [0]:
# https://github.com/openai/gym/blob/master/gym/envs/__init__.py

from gym.envs.registration import register

try:
    register(
        id='FrozenLakeNoSlip-v0',
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'map_name' : '4x4', 'is_slippery': False},
        max_episode_steps=100,
        reward_threshold=0.78, # optimum = .8196
    )
except:
    pass

env_name = 'FrozenLakeNoSlip-v0'
env = gym.make(env_name)

In [0]:
env.observation_space

Discrete(16)

In [0]:
env.action_space

Discrete(4)

In [0]:
type(env.action_space)

gym.spaces.discrete.Discrete

In [0]:
# dunder

class Agent():
    def __init__(self, env):
        self.env = env
        self.actionSize = env.action_space.n
        self.stateSize = env.observation_space.n
        
    def getAction(self, state):
        action = self.env.action_space.sample()      
        return action

agent = Agent(env)

In [0]:
for ep in range(20):
    state = env.reset()
    done = False
    while not done:
        action = agent.getAction(state)
        state, reward, done, info = env.step(action)
        env.render()
        time.sleep(0.05)
        clear_output(wait=True)
        
env.close()

  (Down)
SFFF
FHFH
FFFH
HFFG


# Q- Learning

In [0]:
class Agent():
    def __init__(self, env, discountRate=0.97, learningRate=0.01):
        self.env = env
        self.actionSize = env.action_space.n
        self.stateSize = env.observation_space.n
        
        self.explorationRate = 1.0
        self.discountRate = discountRate
        self.learningRate = learningRate
        
        self.qTable = 1e-4*np.random.random([self.stateSize, self.actionSize])
        
    def getAction(self, state):
        qState = self.qTable[state]
        actionGreedy = np.argmax(qState)
        actionRandom = self.env.action_space.sample()
        if random.random() < self.explorationRate:
            action = actionRandom
        else:
            action =  actionGreedy
        return action

    def train(self, state, action, nextState, reward, done):
        if done:
            qNext = np.zeros([self.actionSize])
        else:
            qNext = self.qTable[nextState]
        qTarget = reward + self.discountRate*np.max(qNext)
        qUpdate = qTarget - self.qTable[state, action]
        self.qTable[state,action] += self.learningRate*qUpdate
        
        if done:
            self.explorationRate *= 0.99
    
agent = Agent(env)

In [0]:
totalReward = 0
for ep in range(100):
    state = env.reset()
    done = False
    while not done:
        action = agent.getAction(state)
        nextState, reward, done, info = env.step(action)
        agent.train(state, action, nextState, reward, done)
        
        print(f's:{state}, a:{action}')
        state = nextState
        
        totalReward += reward
        print(f'Ep:{ep}, Goal:{totalReward}, Explore:{agent.explorationRate}')
        
        env.render()
        print(agent.qTable)
        
        time.sleep(0.05)
        clear_output(wait=True)
        
env.close()

s:6, a:0
Ep:99, Goal:0.0, Explore:0.36603234127322926
  (Left)
SFFF
FHFH
FFFH
HFFG
[[4.56373783e-05 4.76309599e-05 8.27070632e-05 6.77901366e-05]
 [8.29597461e-05 2.68261425e-05 4.78174290e-05 7.66472591e-05]
 [4.58029624e-05 4.28410815e-05 2.64959878e-05 7.77015882e-05]
 [6.87247844e-05 4.40637638e-05 9.42116017e-05 1.22319975e-05]
 [6.79293914e-06 1.54744172e-05 5.83241975e-05 4.71801310e-05]
 [6.67256812e-05 6.20912576e-05 7.64222127e-05 3.82215709e-05]
 [3.81417390e-05 4.56575573e-05 7.04261058e-05 4.55172491e-05]
 [1.80781869e-05 9.03906352e-05 5.01316292e-05 5.57440296e-05]
 [2.76412155e-05 2.34144135e-05 9.00821299e-05 3.64128380e-05]
 [8.79972931e-05 6.62768269e-05 6.46278819e-05 6.79789016e-05]
 [1.15808356e-05 6.09590779e-05 1.29434005e-05 2.06453306e-05]
 [8.24784592e-05 3.93622745e-05 4.56483395e-05 9.57306360e-05]
 [9.87483117e-05 9.53857510e-05 2.55635941e-05 5.76277423e-05]
 [2.19361763e-05 1.51547245e-05 5.51616700e-05 4.68155956e-05]
 [8.51423092e-05 1.26757567e-05 1.1

# Q-Learning with Neural Network

In [0]:
import tensorflow as tf

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [0]:
class Agent():
    def __init__(self, env, discountRate=0.97, learningRate=0.01):
        self.env = env
        self.actionSize = env.action_space.n
        self.stateSize = env.observation_space.n
        
        self.explorationRate = 1.0
        self.discountRate = discountRate
        self.learningRate = learningRate
        
        tf.reset_default_graph()
        self.stateIn = tf.placeholder(tf.int32, shape=[1])
        self.actionIn = tf.placeholder(tf.int32, shape=[1])
        self.targetIn = tf.placeholder(tf.float32, shape=[1])
        
        self.state = tf.one_hot(self.stateIn, depth=self.stateSize)
        self.action = tf.one_hot(self.actionIn, depth=self.actionSize)
        
        self.qState = tf.layers.dense(self.state, units=self.actionSize, name='QTable')
        self.qAction = tf.reduce_sum(tf.multiply(self.qState, self.action), axis=1)
        
        self.loss = tf.reduce_sum(tf.square(self.targetIn - self.qAction))
        self.optimizer = tf.train.AdamOptimizer(self.learningRate).minimize(self.loss)
        
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        
    def getAction(self, state):
        qState = self.sess.run(self.qState, feed_dict={self.stateIn:[state]})
        actionGreedy = np.argmax(qState)
        actionRandom = self.env.action_space.sample()
        if random.random() < self.explorationRate:
            action = actionRandom
        else:
            action =  actionGreedy
        return action

    def train(self, state, action, nextState, reward, done):
        if done:
            qNext = np.zeros([self.actionSize])
        else:
            qNext = self.sess.run(self.qState, feed_dict={self.stateIn:[nextState]})
        qTarget = reward + self.discountRate*np.max(qNext)
        
        feed = {
            self.stateIn: [state],
            self.actionIn: [action],
            self.targetIn: [qTarget]
        }
        self.sess.run(self.optimizer, feed_dict=feed)
        
        if done:
            self.explorationRate *= 0.99
            
    def __del__(self):
        self.sess.close()
    
agent = Agent(env)

In [0]:
totalReward = 0
for ep in range(100):
    state = env.reset()
    done = False
    while not done:
        action = agent.getAction(state)
        nextState, reward, done, info = env.step(action)
        agent.train(state, action, nextState, reward, done)
        
        print(f's:{state}, a:{action}')
        state = nextState
        
        totalReward += reward
        print(f'Ep:{ep}, Goal:{totalReward}, Explore:{agent.explorationRate}')
        
        env.render()
        
        with tf.variable_scope('QTable', reuse=True):
            weights = agent.sess.run(tf.get_variable('kernel'))
            print(weights)
        
        time.sleep(0.05)
        clear_output(wait=True)
        
env.close()

s:8, a:1
Ep:99, Goal:8.0, Explore:0.36603234127322926
  (Down)
SFFF
FHFH
FFFH
HFFG
[[ 0.24509813  0.16395125  0.20618448  0.02229135]
 [ 0.18115844 -0.51777035  0.14276125 -0.17493106]
 [ 0.17200133 -0.02739375  0.3345015  -0.0501372 ]
 [ 0.2293277  -0.43240154  0.26797414 -0.06178652]
 [ 0.25584513  0.1722035  -0.50136817  0.09133032]
 [-0.2533045   0.12269038  0.50213706  0.297525  ]
 [-0.27757084 -0.04342653 -0.06098402 -0.07776279]
 [-0.20415747 -0.17382884  0.16484326 -0.27588147]
 [ 0.29695252 -0.8070774   0.51242816  0.13374288]
 [ 0.32144007  0.18435301  0.39254153 -0.7583182 ]
 [ 0.00279046  0.20622282 -0.2573882  -0.13300154]
 [ 0.43928647 -0.51543176  0.10943538 -0.09816831]
 [-0.09062845 -0.10944542 -0.10986134  0.2589336 ]
 [-0.6603931   0.10617203  0.51853764  0.16442236]
 [ 0.34697786  0.17438534  0.48953643  0.18942189]
 [ 0.31645572 -0.3779943   0.27900732  0.4119057 ]]
